source: [LeetCode]()

# 🧠 Quick Review — LeetCode 146: LRU Cache  
**Difficulty:** Medium  
**Topic:** Design / HashMap + Doubly Linked List  

---

## 📌 Problem Summary  
Design a data structure that implements an **LRU (Least Recently Used) Cache** with:

- `get(key)` → returns value or `-1`  
- `put(key, value)` → inserts/updates key  
- When capacity is exceeded → evict the **least recently used** key  
- Both operations must run in **O(1)** average time  

---

## 🔍 Example Walkthrough  
LRUCache(2)

put(1,1) → {1=1} 

put(2,2) → {1=1, 2=2} 

get(1) → 1 (1 becomes most recently used) 

put(3,3) → evict key=2 

put(4,4) → evict key=1 

get(3) → 3 

get(4) → 4 


---

## 🚀 Key Idea  
To guarantee **O(1)** operations:

### Use:
- **HashMap** → direct access to nodes by key  
- **Doubly Linked List** → maintain usage order  
  - Head = most recently used  
  - Tail = least recently used  

### Why doubly linked list?  
Because you must:
- Move a node to the front in O(1)  
- Remove the tail in O(1)  
- Delete middle nodes in O(1)

A singly linked list would require O(n) removal.

---

## 📝 Short Approach (Interview-Ready)

1. Maintain a **DLL** with:
   - `head.next` as most recently used  
   - `tail.prev` as least recently used  
2. Maintain a **hashmap `{key: node}`**  
3. `get(key)`:
   - If key missing → return -1  
   - Move node to **front** (most recently used)  
4. `put(key, value)`:
   - If key exists → update and move to front  
   - Else:
     - Create a new node and insert at front  
     - Add to hashmap  
     - If size exceeds capacity:
       - Remove tail node (LRU)  
       - Remove from hashmap  

Time Complexity → **O(1)**  
Space Complexity → **O(capacity)**  

---

## 🧑‍💻 Template Code (Clean, Standard Implementation)
```python
class Node:
    def __init__(self, key, val):
        self.key = key
        self.val = val
        self.prev = None
        self.next = None

class LRUCache:

    def __init__(self, capacity):
        self.cap = capacity
        self.map = {}
        
        # dummy head and tail
        self.head = Node(0, 0)
        self.tail = Node(0, 0)
        self.head.next = self.tail
        self.tail.prev = self.head

    # helper: remove node
    def _remove(self, node):
        node.prev.next = node.next
        node.next.prev = node.prev

    # helper: insert after head (most recently used)
    def _insert_front(self, node):
        node.next = self.head.next
        node.prev = self.head
        self.head.next.prev = node
        self.head.next = node

    def get(self, key):
        if key not in self.map:
            return -1
        node = self.map[key]
        self._remove(node)
        self._insert_front(node)
        return node.val

    def put(self, key, value):
        if key in self.map:
            self._remove(self.map[key])

        node = Node(key, value)
        self._insert_front(node)
        self.map[key] = node

        if len(self.map) > self.cap:
            # remove LRU (tail.prev)
            lru = self.tail.prev
            self._remove(lru)
            del self.map[lru.key]


# My solution

In [ ]:
class LRU_Node:
    def __init__(self, key, val):
        self.key = key
        self.val = val
        self.prev = None
        self.next = None

class LRUCache(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.hm = {}
        self.n = 0
        self.head = None
        self.tail = None

    def _remove(self, node):
        if node.prev:
            node.prev.next = node.next
        else:
            self.head = node.next

        if node.next:
            node.next.prev = node.prev
        else:
            self.tail = node.prev

        node.prev = None
        node.next = None

    def _add_to_tail(self, node):
        if not self.tail:
            self.head = self.tail = node
        else:
            self.tail.next = node
            node.prev = self.tail
            self.tail = node

    def get(self, key):
        if key not in self.hm:
            return -1

        node = self.hm[key]
        self._remove(node)
        self._add_to_tail(node)
        return node.val

    def put(self, key, value):
        if key in self.hm:
            node = self.hm[key]
            node.val = value
            self._remove(node)
            self._add_to_tail(node)
            return

        if self.n == self.capacity:
            # evict LRU
            del_node = self.head
            self._remove(del_node)
            del self.hm[del_node.key]
            self.n -= 1

        new_node = LRU_Node(key, value)
        self._add_to_tail(new_node)
        self.hm[key] = new_node
        self.n += 1
